<a href="https://colab.research.google.com/github/MdNiamul/DataHioding/blob/main/Image_hiding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
"""
Steganography Image Decoder (LSB) — Colab-friendly
- Upload an image and extract hidden data.
- Supports length-header or sentinel mode.
- Optional Hamming(7,4) ECC.
"""

import zlib
from typing import List
from PIL import Image
from google.colab import files

# ------------------------------- Bit utils -----------------------------------

def bits_to_int(bits: List[int], endian: str = "big") -> int:
    val = 0
    if endian == "big":
        for b in bits:
            val = (val << 1) | (b & 1)
    else:
        for i, b in enumerate(bits):
            val |= ((b & 1) << i)
    return val

def bits_to_bytes(bits: List[int]) -> bytes:
    if len(bits) % 8 != 0:
        bits = bits[: len(bits) - (len(bits) % 8)]
    by = bytearray()
    for i in range(0, len(bits), 8):
        by.append(bits_to_int(bits[i:i+8], endian="big"))
    return bytes(by)

def bytes_to_bits(data: bytes) -> List[int]:
    out = []
    for byte in data:
        for i in range(7, -1, -1):
            out.append((byte >> i) & 1)
    return out

# -------------------------- Hamming(7,4) ECC utils ---------------------------

def _hamming74_syndrome(code: List[int]) -> int:
    p1 = code[0] ^ code[2] ^ code[4] ^ code[6]
    p2 = code[1] ^ code[2] ^ code[5] ^ code[6]
    p3 = code[3] ^ code[4] ^ code[5] ^ code[6]
    return (p1 << 2) | (p2 << 1) | p3

def hamming74_decode(bits: List[int]) -> List[int]:
    out = []
    for i in range(0, len(bits) - (len(bits) % 7), 7):
        block = bits[i:i+7]
        syn = _hamming74_syndrome(block)
        if syn != 0:
            pos = syn - 1
            if 0 <= pos < 7:
                block[pos] ^= 1
        out.extend([block[2], block[4], block[5], block[6]])
    return out

# ------------------------------- LSB extract ---------------------------------

def extract_lsb_bits(img: Image.Image, bits_per_channel: int = 1, channels: str = "RGB", lsb_order: str = "lsb") -> List[int]:
    channels = channels.upper()
    allowed = {"R":0, "G":1, "B":2}
    idxs = [allowed[c] for c in channels if c in allowed]
    if not idxs:
        raise ValueError("channels must include at least one of R,G,B")
    if img.mode != "RGB":
        img = img.convert("RGB")

    w, h = img.size
    px = img.load()
    out_bits: List[int] = []
    for y in range(h):
        for x in range(w):
            r, g, b = px[x, y]
            vals = (r, g, b)
            for ch in idxs:
                v = vals[ch]
                if lsb_order == "lsb":
                    for k in range(bits_per_channel):
                        out_bits.append((v >> k) & 1)
                else:
                    for k in reversed(range(bits_per_channel)):
                        out_bits.append((v >> k) & 1)
    return out_bits

# ------------------------------ Decoders -------------------------------------

def decode_length_header(bitstream: List[int], ecc: str = "none", endian: str = "big") -> bytes:
    if len(bitstream) < 64:
        raise ValueError("Bitstream too short for length header")
    length_bits = bitstream[:32]
    crc_bits = bitstream[32:64]
    payload_bits = bitstream[64:]

    length = bits_to_int(length_bits, endian=endian)
    declared_crc = bits_to_int(crc_bits, endian=endian)

    if ecc == "hamming":
        payload_bits = hamming74_decode(payload_bits)

    needed = length * 8
    if len(payload_bits) < needed:
        raise ValueError("Not enough bits for declared payload length")
    data = bits_to_bytes(payload_bits[:needed])
    calc_crc = zlib.crc32(data) & 0xFFFFFFFF
    if calc_crc != declared_crc:
        raise ValueError(f"CRC mismatch (got {calc_crc:#010x}, expected {declared_crc:#010x})")
    return data

def decode_sentinel(bitstream: List[int], sentinel: bytes, ecc: str = "none") -> bytes:
    if ecc == "hamming":
        bitstream = hamming74_decode(bitstream)
    sent_bits = bytes_to_bits(sentinel)
    for i in range(0, len(bitstream) - len(sent_bits) + 1):
        if bitstream[i:i+len(sent_bits)] == sent_bits:
            return bits_to_bytes(bitstream[:i])
    raise ValueError("Sentinel not found in bitstream")

# ------------------------------- Colab runner --------------------------------

def run_colab_decoder():
    print("📤 Upload the image with hidden data:")
    uploaded = files.upload()
    if not uploaded:
        print("No file uploaded!")
        return

    img_path = list(uploaded.keys())[0]
    img = Image.open(img_path)

    # Customize options here
    mode = "length"         # "length" or "sentinel"
    bits = 1
    channels = "RGB"
    ecc = "none"
    sentinel = "EOF!"       # only used in sentinel mode
    lsb_order = "lsb"
    endian = "big"
    output_file = "extracted_data.bin"

    print(f"🔍 Decoding {img_path} ...")
    bits_stream = extract_lsb_bits(img, bits_per_channel=bits, channels=channels, lsb_order=lsb_order)

    if mode == "length":
        data = decode_length_header(bits_stream, ecc=ecc, endian=endian)
    else:
        data = decode_sentinel(bits_stream, sentinel=sentinel.encode("utf-8"), ecc=ecc)

    with open(output_file, "wb") as f:
        f.write(data)

    print(f"[+] Extracted {len(data)} bytes -> {output_file}")
    files.download(output_file)

# Run the decoder
run_colab_decoder()


📤 Upload the image with hidden data:


In [ ]:
!python colab_kernel_launcher.py -i hidden.png --out secret.txt --mode length --bits 1 --channels RGB --ecc none


python3: can't open file '/content/colab_kernel_launcher.py': [Errno 2] No such file or directory


In [ ]:
!pip install pillow
